In [ ]:
pip install modelscope
pip install accelerate
pip install pdfplumber

In [8]:
!nvidia-smi

Wed Nov 29 15:49:15 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.82.01    Driver Version: 470.82.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:08.0 Off |                    0 |
| N/A   36C    P0    56W / 300W |   8297MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from modelscope import AutoTokenizer, AutoModel, snapshot_download
import torch
model_dir = snapshot_download("ZhipuAI/chatglm3-6b-32k", revision = "v1.0.0")
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)

# model = AutoModel.from_pretrained(model_dir,load_in_8bit=True,trust_remote_code=True).cuda()
model = AutoModel.from_pretrained(model_dir,trust_remote_code=True).quantize(8).cuda()
# model = AutoModel.from_pretrained(model_dir,trust_remote_code=True).half().cuda()
model = model.eval()
response, history = model.chat(tokenizer, "你好", history=[])
print(response)
response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)
print(response)

2023-11-29 15:32:57,219 - modelscope - INFO - PyTorch version 2.0.1+cu117 Found.
2023-11-29 15:32:57,223 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2023-11-29 15:32:57,259 - modelscope - INFO - Loading done! Current index file version is 1.9.1, with md5 39fae4d454abcef3e7701cec25721492 and a total number of 924 components indexed
/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-29 15:32:58,345 - modelscope - INFO - Use user-specified model revision: v1.0.0
Loading checkpoint shards: 100%|██████████| 7/7 [00:12<00:00,  1.83s/it]
Failed to load cpm_kernels:No module named 'cpm_kernels'


你好！很高兴见到你，欢迎问我任何问题。
晚上睡不着可能是由多种原因引起的，如压力、焦虑、抑郁、身体不适等。以下是一些建议，希望能帮助你改善睡眠：

1. 建立规律的作息时间：每天尽量按时上床和起床，让身体适应规律的作息。

2. 创造舒适的睡眠环境：确保床垫和枕头舒适，房间保持适当的温度、湿度和安静。避免在床上看电视、玩手机等容易让人兴奋的活动。

3. 避免摄入刺激性食物和饮料：晚上避免摄入咖啡因、尼古丁和酒精等刺激性物质，这些物质会让人更难入睡。

4. 进行适当的放松活动：睡前进行一些放松的活动，如深呼吸、瑜伽、冥想等，有助于减轻压力和焦虑，促进睡眠。

5. 增加白天户外活动：白天进行适量的户外活动，如散步、跑步、骑自行车等，有助于调节睡眠。

6. 尝试睡前放松技巧：有些人发现睡前进行渐进性肌肉松弛、正念冥想等技巧有助于入睡。

7. 如有需要，寻求专业帮助：如果长期存在睡眠问题，可能需要寻求专业医生的帮助。

希望这些建议能帮助你改善睡眠。如果还有其他问题，请随时问我。


In [4]:

import pdfplumber as ppl
import gradio as gr
from pathlib import Path



def pdf_extract(path, mid_pos=300):
    all_pages = []
    all_str = ""
    pdf = ppl.open(path)
    for page in pdf.pages:
        page_words = page.extract_words()
        page_sentences = []
        for word in page_words:
            page_sentences.append([word['text'], round(word['x0']), round(word['x1']), round(word['top'])] )
        sentences_sorted = sorted(page_sentences, key=lambda x:x[3])
        sentences_sorted
        sentences_left = []
        sentences_right = []

        for c in sentences_sorted:
            if c[1]< mid_pos:
                sentences_left.append(c)
            else:
                sentences_right.append(c)


        # 左边按行分开
        tmp = []
        res_l = []
        for i, sen in enumerate(sentences_left):
            if i+1>len(sentences_left)-1:
                break

            if sen[2]<mid_pos and sentences_left[i+1][2]>mid_pos:
                res_l.append(tmp)
                tmp=[]
            else:
                tmp.append(sen)
        if tmp:
            res_l.append(tmp) 

        row_num = len(res_l)

        # 获取分隔位置
        sge_index = []
        index = 0
        for l in res_l:
        # 跳过第一行，只要n-1个位置即可
            index += 1
            if index == 1:
                continue
            sge_index.append(l[0][-1])

        # 前后插入
        sge_index.insert(0, -9999)
        sge_index.append(9999)

        # 右边按行分开
        res_r = [[] for i in range(row_num)]
        index = 0
        for i, sen in enumerate(sentences_right):

            if sen[3]<=sge_index[1] and sen[3]>sge_index[0]:
                res_r[index].append(sen)
            elif sen[3]<=sge_index[2] and sen[3]>sge_index[1]:
                res_r[index+1].append(sen)
                sge_index.pop(0)
                index += 1
            else:
                sge_index.pop(0)
                index += 1

        # 合并左右
        final = []
        for i in range(row_num):
            tmp_s = ""
            for j in res_l[i]:
                tmp_s += j[0]
            for k in res_r[i]:
                tmp_s += k[0]
            final.append(tmp_s)

        # 把当前页面放入所有页面结果
    #     all_pages.append(final)
    # return all_pages
    
        all_str += "\n".join(final)
    return all_str

In [2]:
import pdfplumber as ppl
import gradio as gr
from pathlib import Path

def extract_columns(filename, page_center=290):
    def _is_chinese(word):
        for ch in word:
            if '\u4e00' <= ch <= '\u9fff':
                return True
        return False
    

        # 一个经验值，即pdf文档页面中间的x坐标，用于把字符分成左右两部分。
        
    pdf = ppl.open(filename)
    paras = []
    for page in pdf.pages:
        page_words = page.extract_words()
        # x0：单词的第一个字符的x坐标
        # x1：单词的最后一个字符的x坐标
        # top：单词的第一个字符的y坐标
        sentence = [[word['text'], round(word['x0']), round(word['x1']), round(word['top'])] for word in page_words]
        rows = []
        tmp = []
        for word in sentence:
            # 前后单词y坐标相差大于4（这里4也是一个经验值，一般不超过10），说明不是同一行
            if tmp and abs(word[-1] - tmp[-1][-1]) > 4:
                # 另起一行
                rows.append(tmp)
                tmp = []
            tmp.append(word)
        if tmp:
            rows.append(tmp)

        rows_in_page = []
        left_block, right_block = [], []

        for row in rows:
            is_two_columns=False
            row_left, row_right = [], []
            # row是一个二维list，表示一行的内容，形式如：[[text, x0, x1, top], [[text, x0, x1, top]], ...]
            for i in range(len(row)):

                # 用单词最后一个字符的x坐标判断属于左边还是右边
                if row[i][2]<page_center:
                    row_left.append(row[i])
                else:
                    row_right.append(row[i])
                # 前后两个单词相距超过10（说明中间有较长的空白），并且前后两个单词分别属于左右两边，则说明这一行是列式布局
                if i>0 and row[i][1]- row[i-1][2]>10 and row[i][1]>page_center and row[i-1][2]<page_center:
                    is_two_columns=True
            if is_two_columns:
                left_block.extend(row_left)
                right_block.extend(row_right)
            else:
                # 把之前的left和right添加，先左后右
                if left_block:
                    rows_in_page.extend(left_block)
                if right_block:
                    rows_in_page.extend(right_block)

                # 这一行不是列式布局，则正常添加当前行
                rows_in_page.extend(row)

                # 清空left和right
                left_block=[]
                right_block=[]

        # 中英文字符空格处理
        txt = [x[0] if _is_chinese(x[0]) else ' '+x[0] for x in rows_in_page]
        paras.extend(txt)

    # chunks = []
    # chunk = ''
    # for para in paras:
    #     if len_func(chunk + para) < chunk_size:
    #         chunk += para
    #     else:
    #         chunks.append(chunk)
    #         chunk = para
    # if chunk:
    #     chunks.append(chunk)
    # return chunks
    return "".join(paras)

In [4]:
file_res = pdf_extract('/mnt/workspace/askpdf/玉米收储政策改革及其效应分析.pdf')

In [3]:
file_res = extract_columns('/mnt/workspace/askpdf/Adversaria.pdf')

In [5]:
with torch.no_grad():
    response, history = model.chat(tokenizer, f"请帮我总结这篇文章的内容：{file_res}", history=[])
print(response)

本文分析了玉米收储政策改革及其效应。主要结论有：

1. 玉米收储政策调整后，农户结构调整阵痛不可避免。农民决定是否种植玉米受市场价格和补贴影响。预计2017年玉米种植面积将减少。

2. 农民依然面临卖粮难风险。2016年秋粮上市后，农民卖粮困难，出现排队卖粮现象。这需要完善市场结构，使市场决定价格，才能化解。 

3. 玉米价格下跌对下游企业和养殖业有利，对农民不利。政府应加快完善改革方案，不断创新农民增收思路，优化玉米收购市场结构，为农民提供更多公共服务。

4. 玉米价格的变动取决于市场供求关系，不再取决于政策。农民种植玉米收益将转向降低成本、集约经营、规模经营等方面。

5. 政府应关注农民收入变化，完善价格支持政策，帮助农民适应玉米价格市场化的新环境。


In [8]:
with torch.no_grad():
    response, history = model.chat(tokenizer, f"文章的第二章讲述了什么？", history=history)
print(response)

文章的第二章主要讲述了玉米临时收储政策效应及其终结。

首先，文章分析了玉米临时收储政策的基本政策内涵，即国家在每年一定的收购价格向农民敞开收购玉米。这一政策在2008年出台，旨在调动农民种粮积极性，增加粮食产量，实现粮食增产、农民增收的目标。

然后，文章分析了玉米临时收储政策所产生的负面效应。主要体现在：

1. 玉米价格的上涨导致下游产业成本急剧提升，玉米加工业开工率不足，玉米深加工企业亏损，进口玉米挤占国产玉米市场。

2. 玉米价格上涨导致东北地区粮食种植结构失衡，农民多种玉米、少种大豆，大豆种植面积减少。

3. 玉米价格上涨导致东北地区粮食种植面积增加，以开荒方式扩大玉米种植规模，造成生态资源环境破坏。

4. 玉米临时收储政策给国家财政带来沉重负担。

最后，文章指出玉米临时收储政策所产生的负面效应已经超过了正面效应，其对下游产业、对种植业结构、对生态资源环境所带来的种种负面影响，以及给国家财政带来的沉重负担，使这一政策难以为继。2015年秋粮上市之际，在国家连续5次提升玉米临储价格之后，首次下调玉米临储价格，释放了缩减玉米供给的信号。2016年，国家取消了玉米临时收储政策，决定实施“市场化收购”加“补贴”的新机制。


In [6]:
with torch.no_grad():
    response, history = model.chat(tokenizer, f"请帮我总结这篇文章的内容：{file_res}"[:1000], history=[])
print(response)

这篇文章主要探讨了深度神经网络（DNNs）在图数据上的对抗攻击和防御问题。由于DNNs在许多应用场景中表现出色，如图像分类、文本生成、生物识别等，因此它们被广泛应用于各种任务。然而，最近的研究表明，尽管DNNs在某些安全关键任务上得到了应用，如自动驾驶、恶意软件检测等，但它们仍然容易受到对抗攻击的影响。尽管已经有一些研究探讨了对抗攻击和防御策略，如自动驾驶领域，但针对图数据的防御策略仍具有挑战性。由于图数据的特殊结构，DNNs在处理图数据时可能变得脆弱。因此，研究图数据上的对抗攻击和防御策略对于确保图数据的可靠性和安全性至关重要。


In [7]:
with torch.no_grad():
    response, history = model.chat(tokenizer, '这篇论文提出了什么方法', history=history)
print(response)

抱歉，我在之前的回答中犯了一个错误。经过再次查找，我发现这篇论文并没有提出具体的方法。这篇论文对深度神经网络（DNNs）在图数据上的对抗攻击和防御问题进行了调查，指出了DNNs在处理图数据时可能存在的安全问题，但并未提出相应的解决方案。在此向您表示歉意。
